In [0]:
from pyspark.sql import functions as F

In [0]:
filepath = "s3://full-stack-bigdata-datasets/Big_Data/youtube_playlog.csv"

In [0]:
playlog = (spark.read.format('csv')\
             .option('header', 'true')\
             .option('inferSchema', 'true')\
             .load(filepath))
playlog.show(5)

+----------+----+-----------+
| timestamp|user|       song|
+----------+----+-----------+
|1392387533|   0|t1l8Z6gLPzo|
|1392387538|   1|t1l8Z6gLPzo|
|1392387556|   2|t1l8Z6gLPzo|
|1392387561|   3|we5gzZq5Avg|
|1392387566|   4|we5gzZq5Avg|
+----------+----+-----------+
only showing top 5 rows



In [0]:
playlog.printSchema()

root
 |-- timestamp: integer (nullable = true)
 |-- user: integer (nullable = true)
 |-- song: string (nullable = true)



In [0]:
playlog.describe().toPandas()

,summary,timestamp,user,song
0,count,25739537,25739537,25739537
1,mean,1.4427006561045842E9,12697.352275450798,2.532571778181818E8
2,stddev,3.443284872371195E7,13094.065905828476,8.334645614940468E8
3,min,-139955897,0,---AtpxbkaE
4,max,1554321113,45903,zzzcFgRMY6c


In [0]:
import pandas as pd

pd.DataFrame.from_dict(
  {c: playlog.filter(playlog[c].isNull()).count() for c in playlog.columns},
  orient='index', columns=['missing values']
).T

,timestamp,user,song
missing values,0,0,0


In [0]:
playlog.count() == playlog.dropDuplicates().count()

Out[7]: False

In [0]:
(playlog.count() - playlog.dropDuplicates().count())

Out[8]: 123651

In [0]:
playlog.orderBy('timestamp').show(5)

+----------+----+-----------+
| timestamp|user|       song|
+----------+----+-----------+
|-139955897|   4|nRa-eGzpT6o|
|1392387533|   0|t1l8Z6gLPzo|
|1392387537|  70|VJ6ofd0pB_c|
|1392387537|  22|Q24VZL8wpOM|
|1392387538|   1|t1l8Z6gLPzo|
+----------+----+-----------+
only showing top 5 rows



In [0]:
playlog.where(playlog['timestamp'] < 0).count()

Out[10]: 1

In [0]:
playlog.where(playlog['timestamp'] < 0).collect()

Out[11]: [Row(timestamp=-139955897, user=4, song='nRa-eGzpT6o')]

In [0]:
playlog_processed = (playlog.dropDuplicates() \
                     .filter(~(playlog.timestamp < 0)))
playlog_processed.count()

Out[12]: 25615885

In [0]:
output_path = 's3://full-stack-bigdata-datasets/Big_Data/playlog_processed_student.parquet'

playlog_processed.write \
  .parquet(output_path, mode='overwrite')

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-3566053367026343>:3
      1 output_path = 's3://full-stack-bigdata-datasets/Big_Data/playlog_processed_student.parquet'
----> 3 playlog_processed.write \
      4   .parquet(output_path, mode='overwrite')

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1655, in DataFrameWriter.parquet(self, path, mode, partitionBy, compression)
   1653     self.partitionBy(partitionBy)
   1654 self._set_opts(compression=compression)
-> 1655 self._jwrite.parquet(path